In [1]:
import os
import sys
import joblib

sys.path.append(os.getcwd().replace(f"\classification", ""))

print(f"Working directory: {os.getcwd()}")
for path in sys.path:
    print(path)

Working directory: C:\git\MachineLearning\Project\InteractionsClassification\src\classification
C:\git\MachineLearning\Project\InteractionsClassification\src\classification
C:\Users\luisp\AppData\Local\Programs\Python\Python37\python37.zip
C:\Users\luisp\AppData\Local\Programs\Python\Python37\DLLs
C:\Users\luisp\AppData\Local\Programs\Python\Python37\lib
C:\Users\luisp\AppData\Local\Programs\Python\Python37
C:\git\MachineLearning\Project\InteractionsClassification\venv

C:\git\MachineLearning\Project\InteractionsClassification\venv\lib\site-packages
C:\git\MachineLearning\Project\InteractionsClassification\venv\lib\site-packages\setuptools-40.8.0-py3.7.egg
C:\git\MachineLearning\Project\InteractionsClassification\venv\lib\site-packages\IPython\extensions
C:\Users\luisp\.ipython
C:\git\MachineLearning\Project\InteractionsClassification\src


In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

import config_loader
from dataset import dataset_loader

dataset = dataset_loader.load_dataset(config_loader.read_config('../../'))

# Create test dataset
from sklearn.model_selection import train_test_split

X_sample, X_discard, y_sample, y_discard = train_test_split(dataset['features'], dataset['targets'], random_state=33,
                                                            test_size=0.5)
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, random_state=33, test_size=0.2)



Configuration READY
Creating dataset from STRING...
Creating features...
Features READY
Creating targets...
Reading Reactome unique interactions...
Reactome interactions READY
Added index to targets.


In [3]:
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (427858, 7)
y_train shape:  (427858,)
X_test shape:  (106965, 7)
y_test shape:  (106965,)


In [9]:
# Create model
name = "stochastic gradient descent classifier"
clf = joblib.load("models/" + name.replace(" ", "_") + ".pkl")

In [22]:
# Utility function to report best scores
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from time import time
import numpy as np

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
    "penalty": ["none", "l1", "l2"],
    "max_iter": [2000]
}

# run randomized search
n_iter_search = 3
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))


ModuleNotFoundError: No module named 'numpý'

In [18]:
# The best combination of parameters is:
random_search.best_params_

{'penalty': 'none', 'max_iter': 1500, 'alpha': 0.001}

In [19]:
# Get the best estimator and store it

import joblib
joblib.dump(random_search.best_estimator_, "sgd_classifier.pkl")
random_search.best_estimator_

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1500, n_iter_no_change=5, n_jobs=None, penalty='none',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
random_search.best_estimator_.feature_importances_

AttributeError: 'SGDClassifier' object has no attribute 'feature_importances_'

In [21]:
report(random_search.cv_results_)

NameError: name 'np' is not defined